In [1]:
import pandas as pd
import numpy as np

# Load the data
data = pd.read_csv(fr'C:\Users\ivane\Desktop\Dissertation\data\index_MSEPRX.csv')

data.head()


,Date,Index,Change,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,2025-01-10,3736.886,-0.66,NaN,NaN,NaN,NaN
1,2025-01-09,3761.777,-0.09,NaN,NaN,NaN,NaN
2,2025-01-08,3765.056,0.28,NaN,NaN,NaN,NaN
3,2025-01-07,3754.615,1.28,NaN,NaN,NaN,NaN
4,2025-01-06,3707.014,-1.36,NaN,NaN,NaN,NaN


In [2]:
data.drop(columns=['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'], inplace=True)

data.head()



,Date,Index,Change
0,2025-01-10,3736.886,-0.66
1,2025-01-09,3761.777,-0.09
2,2025-01-08,3765.056,0.28
3,2025-01-07,3754.615,1.28
4,2025-01-06,3707.014,-1.36
